In [ ]:
import numpy as np

def gcd(a, b):
    while b:
        a, b = b, a % b
    return a

def matrix_inverse(matrix, mod):
    det = int(np.linalg.det(matrix))
    det_inv = pow(det, -1, mod)

    adj_matrix = np.array(matrix).T
    inv_matrix = (det_inv * adj_matrix) % mod

    return inv_matrix

def matrix_from_key(key, n):
    key = key.upper().replace(" ", "")
    matrix = []
    for i in range(0, len(key), n):
        block = [ord(char) - ord('A') for char in key[i:i+n]]
        matrix.append(block)
    return np.array(matrix)

def hill_encrypt(plain_text, key_matrix, mod):
    n = key_matrix.shape[0]
    plain_text = plain_text.upper().replace(" ", "")
    plain_text = [ord(char) - ord('A') for char in plain_text]

    while len(plain_text) % n != 0:
        plain_text.append(23)  # Adding 'X' which is equivalent to 23

    encrypted_text = []
    for i in range(0, len(plain_text), n):
        block = np.array(plain_text[i:i+n])
        encrypted_block = np.dot(block, key_matrix) % mod
        encrypted_text.extend(encrypted_block)

    encrypted_text = [chr(num + ord('A')) for num in encrypted_text]
    return ''.join(encrypted_text)

def hill_decrypt(encrypted_text, key_matrix, mod):
    n = key_matrix.shape[0]
    encrypted_text = encrypted_text.upper().replace(" ", "")
    encrypted_text = [ord(char) - ord('A') for char in encrypted_text]

    decrypted_text = []
    inv_key_matrix = matrix_inverse(key_matrix, mod)
    for i in range(0, len(encrypted_text), n):
        block = np.array(encrypted_text[i:i+n])
        decrypted_block = np.dot(block, inv_key_matrix) % mod
        decrypted_text.extend(decrypted_block)

    decrypted_text = [chr(num + ord('A')) for num in decrypted_text]
    return ''.join(decrypted_text)

# Example usage
key_input = input("Enter the key: ")
message_input = input("Enter the message: ")
mod = 26

key_matrix = matrix_from_key(key_input, 3)  # You can change '3' to the desired matrix size

if key_matrix.shape[0] != key_matrix.shape[1]:
    print("Error: Key matrix should be square.")
else:
    encrypted_text = hill_encrypt(message_input, key_matrix, mod)
    decrypted_text = hill_decrypt(encrypted_text, key_matrix, mod)

    print("Encrypted Text:", encrypted_text)
    print("Decrypted Text:", decrypted_text)


In [ ]:
import math
import string
import sys

import numpy as np
from sympy import Matrix


def menu():
    while True:
        print("---- Hill Cipher ----\n")
        print("1) Encrypt a Message.")
        print("2) Decipher a Message.")
        print("3) Force a Ciphertext (Known Plaintext Attack).")
        print("4) Quit.\n")
        try:
            choice = int(input("Select a function to run: "))
            if 1 <= choice <= 4:
                return choice
            else:
                print("\nYou must enter a number from 1 to 4\n")
        except ValueError:
            print("\nYou must enter a number from 1 to 4\n")
        input("Press Enter to continue.\n")


# Create two dictionaries, english alphabet to numbers and numbers to english alphabet, and returns them
def get_alphabet():
    alphabet = {}
    for character in string.ascii_uppercase:
        alphabet[character] = string.ascii_uppercase.index(character)

    reverse_alphabet = {}
    for key, value in alphabet.items():
        reverse_alphabet[value] = key

    return alphabet, reverse_alphabet


# Get input from the user and checks if respects the alphabet
def get_text_input(message, alphabet):
    while True:
        text = input(message)
        text = text.upper()
        if all(keys in alphabet for keys in text):
            return text
        else:
            print("\nThe text must contain only characters from the english alphabet ([A to Z] or [a to z]).")


# Check if the key is a square in length
def is_square(key):
    key_length = len(key)
    if 2 <= key_length == int(math.sqrt(key_length)) ** 2:
        return True
    else:
        return False


# Create the matrix k for the key
def get_key_matrix(key, alphabet):
    k = list(key)
    m = int(math.sqrt(len(k)))
    for (i, character) in enumerate(k):
        k[i] = alphabet[character]

    return np.reshape(k, (m, m))


# Create the matrix of m-grams of a text, if needed, complete the last m-gram with the last letter of the alphabet
def get_text_matrix(text, m, alphabet):
    matrix = list(text)
    remainder = len(text) % m
    for (i, character) in enumerate(matrix):
        matrix[i] = alphabet[character]
    if remainder != 0:
        for i in range(m - remainder):
            matrix.append(25)

    return np.reshape(matrix, (int(len(matrix) / m), m)).transpose()


# Encrypt a Message and returns the ciphertext matrix
def encrypt(key, plaintext, alphabet):
    m = key.shape[0]
    m_grams = plaintext.shape[1]

    # Encrypt the plaintext with the key provided k, calculate matrix c of ciphertext
    ciphertext = np.zeros((m, m_grams)).astype(int)
    for i in range(m_grams):
        ciphertext[:, i] = np.reshape(np.dot(key, plaintext[:, i]) % len(alphabet), m)
    return ciphertext


# Transform a matrix to a text, according to the alphabet
def matrix_to_text(matrix, order, alphabet):
    if order == 't':
        text_array = np.ravel(matrix, order='F')
    else:
        text_array = np.ravel(matrix)
    text = ""
    for i in range(len(text_array)):
        text = text + alphabet[text_array[i]]
    return text


# Check if the key is invertible and in that case returns the inverse of the matrix
def get_inverse(matrix, alphabet):
    alphabet_len = len(alphabet)
    if math.gcd(int(round(np.linalg.det(matrix))), alphabet_len) == 1:
        matrix = Matrix(matrix)
        return np.matrix(matrix.inv_mod(alphabet_len))
    else:
        return None


# Decrypt a Message and returns the plaintext matrix
def decrypt(k_inverse, c, alphabet):
    return encrypt(k_inverse, c, alphabet)


def get_m():
    while True:
        try:
            m = int(input("Insert the length of the grams (m): "))
            if m >= 2:
                return m
            else:
                print("\nYou must enter a number m >= 2\n")
        except ValueError:
            print("\nYou must enter a number m >= 2\n")


# Force a Ciphertext (Known Plaintext Attack)
def plaintext_attack(c, p_inverse, alphabet):
    return encrypt(c, p_inverse, alphabet)


def main():
    while True:
        # Ask the user what function wants to run
        choice = menu()

        # Get two dictionaries, english alphabet to numbers and numbers to english alphabet
        alphabet, reverse_alphabet = get_alphabet()

        # Run the function selected by the user
        if choice == 1:
            # Asks the user the plaintext and the key for the encryption and checks the input
            plaintext = get_text_input("\nInsert the text to be encrypted: ", alphabet)
            key = get_text_input("Insert the key for encryption: ", alphabet)

            if is_square(key):
                # Get the key matrix k
                k = get_key_matrix(key, alphabet)
                print("\nKey Matrix:\n", k)

                # Get the m-grams matrix p of the plaintext
                p = get_text_matrix(plaintext, k.shape[0], alphabet)
                print("Plaintext Matrix:\n", p)

                input("\nPress Enter to begin te encryption.")
                # Encrypt the plaintext
                c = encrypt(k, p, alphabet)

                # Transform the ciphertext matrix to a text of the alphabet
                ciphertext = matrix_to_text(c, "t", reverse_alphabet)

                print("\nThe message has been encrypted.\n")
                print("Generated Ciphertext: ", ciphertext)
                print("Generated Ciphertext Matrix:\n", c, "\n")
            else:
                print("\nThe length of the key must be a square and >= 2.\n")

        elif choice == 2:
            # Asks the user the ciphertext and the key for the encryption and checks the input
            ciphertext = get_text_input("\nInsert the ciphertext to be decrypted: ", alphabet)
            key = get_text_input("Insert the key for decryption: ", alphabet)

            if is_square(key):
                # Get the key matrix k
                k = get_key_matrix(key, alphabet)

                # Check if the key is invertible and in that case returns the inverse of the matrix
                k_inverse = get_inverse(k, alphabet)

                if k_inverse is not None:
                    # Get the m-grams matrix c of the ciphertext
                    c = get_text_matrix(ciphertext, k_inverse.shape[0], alphabet)

                    print("\nKey Matrix:\n", k)
                    print("Ciphertext Matrix:\n", c)

                    input("\nPress Enter to begin the decryption.")

                    # Decrypt the ciphertext
                    p = decrypt(k_inverse, c, alphabet)

                    # Transform the ciphertext matrix to a text of the alphabet
                    plaintext = matrix_to_text(p, "t", reverse_alphabet)

                    print("\nThe message has been decrypted.\n")
                    print("Generated Plaintext: ", plaintext)
                    print("Generated Plaintext Matrix:\n", p, "\n")
                else:
                    print("\nThe matrix of the key provided is not invertible.\n")
            else:
                print("\nThe key must be a square and size >= 2.\n")

        elif choice == 3:
            # Asks the user the text and the ciphertext to use them for the plaintext attack
            plaintext = get_text_input("\nInsert the plaintext for the attack: ", alphabet)
            ciphertext = get_text_input("Insert the ciphertext of the plaintext for the attack: ", alphabet)

            # Asks the user the length of the grams
            m = get_m()

            if len(plaintext) / m >= m:
                # Get the m-grams matrix p of the plaintext and takes the firsts m
                p = get_text_matrix(plaintext, m, alphabet)
                p = p[:, 0:m]

                # Check if the matrix of the plaintext is invertible and in that case returns the inverse of the matrix
                p_inverse = get_inverse(p, alphabet)

                if p_inverse is not None:
                    # Get the m-grams matrix c of the ciphertext
                    c = get_text_matrix(ciphertext, m, alphabet)
                    c = c[:, 0:m]

                    if c.shape[1] == p.shape[0]:
                        print("\nCiphertext Matrix:\n", c)
                        print("Plaintext Matrix:\n", p)

                        input("\nPress Enter to begin the attack.")

                        # Force the ciphertext provided
                        k = plaintext_attack(c, p_inverse, alphabet)

                        # Transform the key matrix to a text of the alphabet
                        key = matrix_to_text(k, "k", reverse_alphabet)

                        print("\nThe key has been found.\n")
                        print("Generated Key: ", key)
                        print("Generated Key Matrix:\n", k, "\n")
                    else:
                        print("\nThe number of m-grams for plaintext and ciphertext are different.\n")
                else:
                    print("\nThe matrix of the plaintext provided is not invertible.\n")
            else:
                print("\nThe length of the plaintext must be compatible with the length of the grams (m).\n")
        elif choice == 4:
            sys.exit(0)
        input("Press Enter to continue.\n")


if __name__ == '__main__':
    main()

---- Hill Cipher ----

1) Encrypt a Message.
2) Decipher a Message.
3) Force a Ciphertext (Known Plaintext Attack).
4) Quit.

Select a function to run: 1

Insert the text to be encrypted: helloo
Insert the key for encryption: abc

The length of the key must be a square and >= 2.

Press Enter to continue.

---- Hill Cipher ----

1) Encrypt a Message.
2) Decipher a Message.
3) Force a Ciphertext (Known Plaintext Attack).
4) Quit.

Select a function to run: 1

Insert the text to be encrypted: ACT
Insert the key for encryption: GYBNQKURP

Key Matrix:
 [[ 6 24  1]
 [13 16 10]
 [20 17 15]]
Plaintext Matrix:
 [[ 0]
 [ 2]
 [19]]

Press Enter to begin te encryption.

The message has been encrypted.

Generated Ciphertext:  POH
Generated Ciphertext Matrix:
 [[15]
 [14]
 [ 7]] 

Press Enter to continue.

---- Hill Cipher ----

1) Encrypt a Message.
2) Decipher a Message.
3) Force a Ciphertext (Known Plaintext Attack).
4) Quit.

Select a function to run: 2

Insert the ciphertext to be decrypted: PO

KeyboardInterrupt: ignored

In [1]:
# Online Python compiler (interpreter) to run Python online.
# Write Python 3 code in this online editor and run it.
import numpy as np

def convert(s, k):
    ara = []
    for i in range(0, k * k, k):
        row = []
        for j in range(i, i + k):
            row.append(ord(s[j]) - 97)
        ara.append(row)
    return ara

k = int(input('Enter key size: '))
matrix = convert(str(input(f'Enter key string: ')), k)
plain = str(input(f'Enter a plain text which is multiple of {k}: '))


def canDec(matrix) :
  det = np.linalg.det(matrix)
  if det != 0 :
    return True
  else :
    return False

def process(matrix, k) :
  for i in range(k):
    for j in range(k):
      matrix[i][j] = (((matrix[i][j] % 26) + 26) % 26)
  return matrix

def mul(v, matrix, k):
    ans = []
    for i in range(k):
        x = 0
        for j in range(k):
            x += v[j] * matrix[j][i]
        x = ((x % 26) + 26) % 26
        ans.append(x)


    return ans

def encrypt(plain, k,matrix):
  cipher = ""
  for i in range(0, len(plain), k):
    v = []
    for j in range(i, i + k) :
      v.append(ord(plain[j]) - 97)

    temp = mul(v, matrix, k)
    for j in range(len(temp)):
        cipher+=(chr(int(temp[j]) + 97))
  return cipher


def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

def inverse(a, m):
    gcd, x, y = extended_gcd(a, m)
    if gcd != 1:
        raise ValueError("Modular inverse does not exist")
    return (x % m + m) % m


def decrypt(cipher, k, matrix):
    ans = ""
    det = int(np.linalg.det(matrix))
    det = ((det%26) + 26) % 26
    det = inverse(det, 26)
    adj = np.linalg.inv(matrix) * np.linalg.det(matrix)
    adj = np.round(adj, decimals=3)
    adj = process(adj, k)
    for i in range(k):
        for j in range(k):
            adj[i][j] = (adj[i][j] * det) % 26
            adj[i][j]=int(adj[i][j])

    ans += encrypt(cipher, k, adj)
    return ans

if not canDec(matrix) :
  print('Enter a valid key')






matrix = process(matrix, k)
cipher = encrypt(plain, k, matrix)
decrypted_text = decrypt(cipher, k, matrix)




print(f"given plain: {plain}")
print(f"encrypted cipher text: {cipher}")
print(f"cypher text after decryption: {decrypted_text}")















KeyboardInterrupt: ignored

In [4]:
import numpy as np

def convert(s, k):
    ara = []
    for i in range(0, k * k, k):
        row = []
        for j in range(i, i + k):
            row.append(ord(s[j]) - 97)
        ara.append(row)
    return ara

k = int(input('Enter key size: '))
matrix = convert(str(input(f'Enter key string: ')), k)
plain = str(input(f'Enter a plain text which is multiple of {k}: '))


def canDec(matrix) :
  det = np.linalg.det(matrix)
  if det != 0 :
    return True
  else :
    return False

def process(matrix, k) :
  for i in range(k):
    for j in range(k):
      matrix[i][j] = (((matrix[i][j] % 26) + 26) % 26)
  return matrix

def mul(v, matrix, k):
    ans = []
    for i in range(k):
        x = 0
        for j in range(k):
            x += v[j] * matrix[j][i]
        x = ((x % 26) + 26) % 26
        ans.append(x)


    return ans

def encrypt(plain, k,matrix):
  cipher = ""
  for i in range(0, len(plain), k):
    v = []
    for j in range(i, i + k) :
      v.append(ord(plain[j]) - 97)

    temp = mul(v, matrix, k)
    for j in range(len(temp)):
        cipher+=(chr(int(temp[j]) + 97))
  return cipher


def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

def inverse(a, m):
    gcd, x, y = extended_gcd(a, m)
    if gcd != 1:
        raise ValueError("Modular inverse does not exist")
    return (x % m + m) % m


def decrypt(cipher, k, matrix):
    ans = ""
    det = int(np.linalg.det(matrix))
    det = ((det%26) + 26) % 26
    det = inverse(det, 26)
    adj = np.linalg.inv(matrix) * np.linalg.det(matrix)
    adj = np.round(adj, decimals=3)
    adj = process(adj, k)
    for i in range(k):
        for j in range(k):
            adj[i][j] = (adj[i][j] * det) % 26
            adj[i][j]=int(adj[i][j])

    ans += encrypt(cipher, k, adj)
    return ans

if not canDec(matrix) :
  print('Enter a valid key')






matrix = process(matrix, k)
cipher = encrypt(plain, k, matrix)
decrypted_text = decrypt(cipher, k, matrix)




print(f"given plain: {plain}")
print(f"encrypted cipher text: {cipher}")
print(f"cypher text after decryption: {decrypted_text}")














Enter key size: 3
Enter key string: gybnqkurp
Enter a plain text which is multiple of 3: abc
given plain: abc
encrypted cipher text: byo
cypher text after decryption: abc


In [11]:
import numpy as np

def convert(s, k):
    ara = []
    for i in range(0, k * k, k):
        row = []
        for j in range(i, i + k):
            row.append(ord(s[j]) - 97)
        ara.append(row)
    return ara

def canDec(matrix):
    det = np.linalg.det(matrix)
    if det != 0:
        return True
    else:
        return False

def process(matrix, k):
    for i in range(k):
        for j in range(k):
            matrix[i][j] = (((matrix[i][j] % 26) + 26) % 26)
    return matrix

def mul(v, matrix, k):
    ans = []
    for i in range(k):
        x = 0
        for j in range(k):
            x += v[j] * matrix[j][i]
        x = ((x % 26) + 26) % 26
        ans.append(x)
    return ans

def encrypt(plain, k, matrix):
    cipher = ""
    for i in range(0, len(plain), k):
        v = []
        for j in range(i, i + k):
            v.append(ord(plain[j]) - 97)

        temp = mul(v, matrix, k)
        for j in range(len(temp)):
            cipher += (chr(int(temp[j]) + 97))
    return cipher

def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

def inverse(a, m):
    gcd, x, y = extended_gcd(a, m)
    if gcd != 1:
        raise ValueError("Modular inverse does not exist")
    return (x % m + m) % m

def decrypt(cipher, k, matrix):
    ans = ""
    det = int(np.linalg.det(matrix))
    det = ((det % 26) + 26) % 26
    det = inverse(det, 26)
    adj = np.linalg.inv(matrix) * np.linalg.det(matrix)
    adj = np.round(adj, decimals=3)
    adj = process(adj, k)
    for i in range(k):
        for j in range(k):
            adj[i][j] = (adj[i][j] * det) % 26
            adj[i][j] = int(adj[i][j])

    ans += encrypt(cipher, k, adj)
    return ans

k = int(input('Enter key size: '))
matrix = convert(str(input(f'Enter key string: ')), k)
operation = input("Enter 'encrypt' or 'decrypt': ")

# Read input from a text file
file_path = input("Enter the path of the input text file: ")
try:
    with open(file_path, 'r') as file:
        input_text = file.read().strip()
except FileNotFoundError:
    print("Input file not found. Please provide a valid file path.")
    exit(1)

output_file_path = input("Enter the path for the output text file: ")

if not canDec(matrix):
    print('Enter a valid key')
else:
    matrix = process(matrix, k)
    if operation == 'encrypt':
        result_text = encrypt(input_text, k, matrix)
    elif operation == 'decrypt':
        result_text = decrypt(input_text, k, matrix)
    else:
        print("Invalid operation. Please enter 'encrypt' or 'decrypt'")
        exit(1)

    # Write the result to the output file
    with open(output_file_path, 'w') as output_file:
        output_file.write(result_text)

    print("Operation completed successfully. Result written to the output file.")


Enter key size: 3
Enter key string: rrfvsvcct
Enter 'encrypt' or 'decrypt': decrypt
Enter the path of the input text file: /content/outpu.txt
Enter the path for the output text file: /content/New Text Document.txt
Operation completed successfully. Result written to the output file.
